# DESC Stress Drop Inversion

Differential-Evolution-based Spectral Correction (DESC) method for estimating stress drops.

## Workflow
1. Load event spectra and parameters from spectral decomposition
2. Stack spectra by magnitude bins
3. ECS optimization using Differential Evolution
4. Estimate source parameters (corner frequency, stress drop, etc.)
5. Save results

In [ ]:
import os
import sys
import shutil
import time
import numpy as np
from scipy.io import loadmat, savemat

# Add src to path
sys.path.insert(0, os.path.abspath('../src'))

from specprocess_stackmag import specprocess_stackmag
from specprocess_floategf_allEQs import specprocess_floategf_allEQs
from specprocess_sourcepara import specprocess_sourcepara

## Load Data and Configuration

In [ ]:
# Source model definitions
prefix_all = ['brune', 'boatwright', 'doublecorner']  # Don't change
method_all = [1, 5, 11]  # Don't change

# Dataset identifier
name_suffix = 'test'

# Load event spectra from spectral decomposition
data = loadmat(f'evspec_{name_suffix}.mat', squeeze_me=True, struct_as_record=False)

# Convert evspec from MATLAB struct array to list of dicts
evspec_raw = data['evspec']
if not isinstance(evspec_raw, (list, np.ndarray)):
    evspec_raw = [evspec_raw]
elif isinstance(evspec_raw, np.ndarray):
    evspec_raw = evspec_raw.tolist()

evspec = []
for ev_obj in evspec_raw:
    if not isinstance(ev_obj, dict):
        # Convert MATLAB struct to dict
        ev_dict = {field: getattr(ev_obj, field) for field in ev_obj._fieldnames}
    else:
        ev_dict = ev_obj
    evspec.append(ev_dict)

freq = np.asarray(data['freq']).squeeze()
freqlim = np.asarray(data['freqlim']).squeeze()

# Convert distspec to list of dicts
distspec_raw = data['distspec']
if not isinstance(distspec_raw, (list, np.ndarray)):
    distspec_raw = [distspec_raw]
elif isinstance(distspec_raw, np.ndarray):
    distspec_raw = distspec_raw.tolist()

distspec = []
for ds_obj in distspec_raw:
    if not isinstance(ds_obj, dict):
        ds_dict = {field: getattr(ds_obj, field) for field in ds_obj._fieldnames}
    else:
        ds_dict = ds_obj
    distspec.append(ds_dict)

# Convert stspec to list of dicts
stspec_raw = data['stspec']
if not isinstance(stspec_raw, (list, np.ndarray)):
    stspec_raw = [stspec_raw]
elif isinstance(stspec_raw, np.ndarray):
    stspec_raw = stspec_raw.tolist()

stspec = []
for st_obj in stspec_raw:
    if not isinstance(st_obj, dict):
        st_dict = {field: getattr(st_obj, field) for field in st_obj._fieldnames}
    else:
        st_dict = st_obj
    stspec.append(st_dict)

y0 = float(data['y0'])
slopeline = float(data['slopeline'])

# Load parameters
P_obj = loadmat('P.mat', squeeze_me=True, struct_as_record=False)['P']
if not isinstance(P_obj, dict):
    P = {field: getattr(P_obj, field) for field in P_obj._fieldnames}
else:
    P = P_obj

# Load magnitude info
maginfo_obj = loadmat('maginfo.mat', squeeze_me=True, struct_as_record=False)['maginfo']
if not isinstance(maginfo_obj, dict):
    maginfo = {field: getattr(maginfo_obj, field) for field in maginfo_obj._fieldnames}
else:
    maginfo = maginfo_obj

nspecmin = 4

print(f"Loaded {len(evspec)} events")
print(f"Frequency range: {freq[0]:.3f} - {freq[-1]:.3f} Hz ({len(freq)} points)")

## Configure Inversion Parameters

In [ ]:
# Spectral model falloff parameter
falloff = np.arange(2.0, 2.1, 0.1)  # Falloff range (usually fixed at 2.0)

# Magnitude binning
maglim = [0.9, 4.0]  # Mw range (better have higher magnitudes)
d_mag = 0.2          # Mw bin size (0.2-0.3 typical)

# Stress drop search range
ddsig = [0.01, 100]  # [min, max] in MPa

# Minimum events per magnitude bin
min_nspec = 10

# Wave type
iwave = 1  # 1=P-wave, 2=S-wave

# Frequency band for spectral fitting
freqfit = [2, 40]  # Hz

print("Inversion parameters:")
print(f"  Magnitude bins: {maglim[0]}-{maglim[1]} Mw, bin size = {d_mag}")
print(f"  Stress drop searching range: {ddsig[0]}-{ddsig[1]} MPa")
print(f"  Min events per bin: {min_nspec}")
print(f"  Fitting band: {freqfit[0]}-{freqfit[1]} Hz")
print(f"  Wave type: {'P' if iwave == 1 else 'S'}")

## Stack Spectra by Magnitude Bins

In [ ]:
itype = 2  # Don't change
stack_spec = specprocess_stackmag(evspec, nspecmin, maginfo, 1, d_mag)

print(f"Stacked spectra into {len(stack_spec)} magnitude bins")
for i, ss in enumerate(stack_spec):
    print(f"  Bin {i+1}: Mw={ss['mag']:.2f}, N={ss['nspec']} events")

## Run Stress Drop Inversion (Brune Model)

Use Differential Evolution to estimate corner frequencies and stress drops.

In [ ]:
isource = 1  # 1=Brune, 2=Boatwright, 3=Double-corner
imethod = method_all[isource - 1]
prefix = prefix_all[isource - 1]

print(f"Solving for {prefix} model (method={imethod})")
print(f"Frequency band: {freqfit[0]}-{freqfit[1]} Hz")

# Find low-frequency indices for calibration
nlow = np.where((freq >= freqlim[0]) & (freq <= freqlim[1]))[0]

# Run floating EGF inversion
start_time = time.time()

ECS, stack_spec, EGF_all, _, _, _ = specprocess_floategf_allEQs(
    evspec, stack_spec, freqfit, ddsig, maglim, imethod, min_nspec, nlow
)

elapsed = time.time() - start_time
print(f"\nInversion completed in {elapsed:.1f} seconds")

## Compute Source Parameters

Estimate corner frequency, stress drop, seismic moment, radiated energy, etc.

In [ ]:
flow = freqfit[0]
fhigh = freqfit[1]

outdir = f'result-indieq-{prefix}-f{flow}-{fhigh}'
outpara = f'para_ECS_{prefix}.mat'
outfig = f'result-all-{prefix}-f{flow}-{fhigh}'

# Create output directories
os.makedirs(outfig, exist_ok=True)

# Compute source parameters for all events
# Arguments: (outdir, freqfit, evspec, imethod, iwave, ECS, save_plot, show_plot)
para_EGF = specprocess_sourcepara(outdir, freqfit, evspec, imethod, iwave, ECS, 1, 0)

print(f"\nSource parameters computed for {len(para_EGF) if isinstance(para_EGF, list) else 'all'} events")
print(f"Results saved to: {outdir}/")

## Save Results and Organize Output

In [ ]:
# Save all results to MAT file
savemat(outpara, {
    'para_EGF': para_EGF,
    'stack_spec': stack_spec,
    'ECS': ECS,
    'distspec': distspec,
    'stspec': stspec,
    'evspec': evspec
})

# Move PDF plots and parameter file to output directory
pdf_files = [f for f in os.listdir('.') if f.endswith('.pdf')]
for pdf in pdf_files:
    shutil.move(pdf, os.path.join(outfig, pdf))

if os.path.exists(outpara):
    shutil.move(outpara, os.path.join(outfig, outpara))

print(f"\nAll results saved to: {outfig}/")
print(f"  - Parameter file: {outpara}")
print(f"  - Individual event results: {outdir}/")
print("\nDone!")